In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table, join, Column

from astropy.io import ascii
import glob, re

% matplotlib inline

import matplotlib as mpl
from matplotlib import rc

mpl.style.use('seaborn-colorblind')

#To make sure we have always the same matplotlib settings
#(the ones in comments are the ipython notebook settings)

mpl.rcParams['figure.figsize']=(12.0,9.0)    #(6.0,4.0)
mpl.rcParams['font.size']=18               #10 
mpl.rcParams['savefig.dpi']= 200             #72 
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

colours = mpl.rcParams['axes.prop_cycle'].by_key()['color']


## This notebook reads in Keith's abundances and Guy's asteroseismology to produce the tables for the paper.

In [2]:
fname_diff = "../data/abundances/Pope_abundances_diff_arcturus.fits"
fname_diff = "../data/abundtable_ref.fits"
fname_nodiff = "../data/abundances/Pope_abundances_nodiff.fits"
fname_diff = "../data/abundtable.fits"


In [3]:
diff = Table.read(fname_diff)
diff.rename_column('Star','Object')
diff.sort('Object')
nodiff = Table.read(fname_nodiff)
nodiff.sort('Object')

How do abundances compare between the differential and non-differential methods? Not too badly. Many more elements lack differential abundances than absolute abundances so it looks like in the paper we shouldn't use the differential. Good to include in online data though.

In [4]:
# for j, key in enumerate(diff.keys()[1:]):
#     if key[0]=='e':
#         continue
#     plt.figure(j)
#     plt.plot(diff[key],nodiff[key],'.')
#     plt.title(key)

In [5]:
nodiff.keys()

['Object',
 'Ca_Fe',
 'eCa_Fe',
 'Mg_Fe',
 'eMg_Fe',
 'Si_Fe',
 'eSi_Fe',
 'Ti_Fe',
 'eTi_Fe',
 'O_Fe',
 'eO_Fe',
 'Al_Fe',
 'eAl_Fe',
 'Ba_Fe',
 'eBa_Fe',
 'Na_Fe',
 'eNa_Fe',
 'Ni_Fe',
 'eNi_Fe',
 'Mn_Fe',
 'eMn_Fe',
 'Co_Fe',
 'eCo_Fe',
 'Eu_Fe',
 'eEu_Fe',
 'La_Fe',
 'eLa_Fe',
 'Zr_Fe',
 'eZr_Fe',
 'Sr_Fe',
 'eSr_Fe',
 'Zn_Fe',
 'eZn_Fe',
 'Y_Fe',
 'eY_Fe',
 'Cr_Fe',
 'eCr_Fe',
 'V_Fe',
 'eV_Fe',
 'Cu_Fe',
 'eCu_Fe',
 'Sc_Fe',
 'eSc_Fe']

In [6]:
elements = filter(lambda x: x[0]!='e', nodiff.keys()[1:])
# print elements

In [7]:
elements

['Ca_Fe',
 'Mg_Fe',
 'Si_Fe',
 'Ti_Fe',
 'O_Fe',
 'Al_Fe',
 'Ba_Fe',
 'Na_Fe',
 'Ni_Fe',
 'Mn_Fe',
 'Co_Fe',
 'Eu_Fe',
 'La_Fe',
 'Zr_Fe',
 'Sr_Fe',
 'Zn_Fe',
 'Y_Fe',
 'Cr_Fe',
 'V_Fe',
 'Cu_Fe',
 'Sc_Fe']

### Create a string-formatted table merging abundances with their uncertainties appropriately.

In [8]:
alpha = ['Mg','Ti','Si','Ca','Al','V','Ni']
neutron = ['Sr','Y','Zr','Ba','La','Eu']

In [9]:
for ell in neutron:
    print ell

Sr
Y
Zr
Ba
La
Eu


In [10]:
newtab = Table([Column([str(obj).replace('_',' ') for obj in diff['Object']],name='Object')])

# first do the differential elements

for el in alpha:
    col = diff[el]
    ecol = diff['%serr' % el]
    test = Column(['--' if 'nan' in str(col[j]) else '$%.2f \pm %.2f$' % (float(col[j]),float(ecol[j])) for j, x in enumerate(col)],name='[%s/Fe]' % el.replace('_Fe',''))
#     test = Column(['$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=el.replace('_Fe',''))
    newtab.add_column(test)
    
#     print el

# Then do the neutron capture elements
newtab2 = Table([Column([str(obj).replace('_',' ') for obj in nodiff['Object']],name='Object')])

for ell in neutron:
    el = '%s_Fe' % ell
    col = nodiff[el]
    ecol = nodiff['e%s' % el]
    test = Column(['--' if 'nan' in str(col[j]) else '$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name='[%s/Fe]' % el.replace('_Fe',''))
#     test = Column(['$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=el.replace('_Fe',''))

    newtab2.add_column(test)
    
# newtab3 = Table([Column([str(obj).replace('_',' ') for obj in nodiff['Object']],name='Object')])

# for el in elements[15:]:
#     col = nodiff[el]
#     ecol = nodiff['e%s' % el]
#     test = Column(['--' if 'nan' in str(col[j]) else '$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name='[%s/Fe]' % el.replace('_Fe',''))
# #     test = Column(['$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=el.replace('_Fe',''))

#     newtab3.add_column(test)

    
# newtab_diff = Table([diff['Object']])

# for el in elements:
#     col = diff[el]
#     ecol = diff['e%s' % el]
#     test = Column(['--' if 'nan' in str(col[j]) else '%.2f $\pm$ %.2f' % (col[j],ecol[j]) for j, x in enumerate(col)],name=el.replace('_Fe',''))
# #     test = Column(['$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=el.replace('_Fe',''))

#     newtab_diff.add_column(test)


In [11]:
def oxford_join(string_list):
    '''From https://stackoverflow.com/questions/19838976/grammatical-list-join-in-python'''
    if len(string_list) < 1:
        text = ''
    elif len(string_list) == 1:
        text = string_list[0]
    elif len(string_list) == 2:
        text = ' and '.join(string_list)
    else:
        text = ', '.join(string_list)
        text = '{parts[0]}, and {parts[2]}'.format(parts=text.rpartition(', '))  # oxford comma
    return text

## Now produce three LaTeX tables to go into the paper directory.

In [12]:
latexdict = ascii.latex.latexdicts['AA'].copy()
latexdict['tabletype']= 'table*'

caption1 = r'Chemical abundances relative to iron for stars in the red giant sample as determined by BACCHUS,  differential line-by-line comparison to Arcturus, as described in Section~\ref{spectroscopy}, for the elements %s. Dashes indicate elements for which abundances could not be reliably computed.' % oxford_join([el.replace('_Fe','') for el in filter(lambda x: x !='O_Fe',elements[0:8])])
caption2 = r'Chemical abundances relative to iron of neutron capture elements for stars in the red giant sample as determined by BACCHUS, without differential line-by-line comparison to Arcturus, as described in Section~\ref{spectroscopy}, for the elements %s. Dashes indicate elements for which abundances could not be reliably computed.' % oxford_join([el.replace('_Fe','') for el in elements[8:15]])
# caption3 = r'Chemical abundances relative to iron for stars in the red giant sample as determined by BACCHUS, without differential line-by-line comparison to Arcturus, as described in Section~\ref{spectroscopy}, for the elements %s. Dashes indicate elements for which abundances could not be reliably computed.' % oxford_join([el.replace('_Fe','') for el in elements[15:]])

caption1 = caption1+r'The catalogue of abundances for neutron capture elements continues in Table~\ref{elems2}.'
# caption2 = caption2+r'The catalogue of abundances for more elements continues in Table~\ref{elems3}.'


newtab.write('../paper/abundances_1.tex',format='latex',latexdict=latexdict,caption=caption1+'\label{elems1}')
newtab2.write('../paper/abundances_2.tex',format='latex',latexdict=latexdict,caption=caption2+'\label{elems2}')
# newtab3.write('../paper/abundances_3.tex',format='latex',latexdict=latexdict,caption=caption3+'\label{elems3}')

### Now let's also do the stellar properties

In [13]:
fname_merged ="../data/abundances/final_pope_merged.fits"
fname_merged = '../data/spc_final.txt'
fname_mass = '../data/smeartest201018_SeismoFullove02_full_notefftol.ascii'
merged = Table.read(fname_merged,format='ascii')
masses = Table.read(fname_mass,format='ascii')
masses['age'] /= 1000. # Gyr not Myr
masses['age_errp'] /= 1000.
masses['age_errm'] /= 1000.

# fname_spc = "../data/spc_final.txt"
# spc = Table.read(fname_spc,format='ascii')
astero_logg = Table.read('../data/pope_asteroseismology.csv')
# merged = Table.read(fname_merged)
merged.rename_column('RVerr','RV_err')
merged.keys()
merged.sort('Object')
joint = join(merged,astero_logg,keys='Object')
joint = join(joint,masses,keys='Object')

In [14]:
masses.keys()

['Object',
 'massfin',
 'massfin_errp',
 'massfin_errm',
 'radPhot',
 'radPhot_errp',
 'radPhot_errm',
 'logg',
 'logg_errp',
 'logg_errm',
 'age',
 'age_errp',
 'age_errm']

In [15]:
counter = []
counter2 = []
for obj in merged['Object']:
    if obj not in astero_logg['Object']:
#         print obj
        counter.append(obj)
for obj in astero_logg['Object']:
    if obj not in merged['Object']:
#         print obj
        counter2.append(obj)

# print len(counter)
# print len(counter2)
# print counter

In [16]:
for j, obj in enumerate(astero_logg['Object']):
    ind = merged['Object']==obj
    merged[ind]['log_g'] = astero_logg['log_g'][j]
    merged[ind]['log_g_err'] = astero_logg['log_g_err'][j]

In [29]:
newtab = Table([Column([str(obj).replace('_',' ') for obj in joint['Object']],name='Object')])

newkeys = ['RV', 'Teff', 'log_g', 'monh','vsini','SNRe']
names = [r'RV',r'\teff', r'\logg',r'[M/H]',r'$V\sin{i}$',r'SNR']
units = [r'(km/s)', r'(K)', '','','(km/s)','']

for k, key in enumerate(newkeys):
    col = joint[key]
    if key+'_err' in joint.keys():
        ecol = joint['%s_err' % key]
        if 'Teff' in key:
            test = Column(['--' if 'nan' in str(col[j]) else '$%.0f \pm %.0f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=names[k])
        else:
            test = Column(['--' if 'nan' in str(col[j]) else '$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=names[k])
    else:
        test = Column(['%.1f' % (col[j]) for j, x in enumerate(col)],name=names[k])
    
    newtab.add_column(test)
    
amaliekeys = ['massfin','radPhot','age']
amalienames = ['Mass', 'Radius','Age']
amalieunits = ['(\msun)','(\\rsun)','(Gyr)']

for k, key in enumerate(amaliekeys):
    col = joint[key]
    ecolp = joint['%s_errp' % key]
    ecolm = joint['%s_errm' % key]
    test = Column(['--' if 'nan' in str(col[j]) else '$%.2f^{+%.2f}_{-%.2f}$' % (col[j],ecolp[j],ecolm[j]) for j, x in enumerate(col)],name=amalienames[k])
    
    newtab.add_column(test)


In [30]:
amalieunits

['(\\msun)', '(\\rsun)', '(Gyr)']

In [31]:
# newtab

In [32]:
latexdict = ascii.latex.latexdicts['AA'].copy()
latexdict['tabletype']= 'table*'
latexdict['units'] = dict(zip(names+amalienames,units+amalieunits))

caption = r'Fundamental stellar parameters for the red giant sample as determined jointly by asteroseismology (asteroseismic \logg; Section~\ref{asteroseismology}) and spectroscopy (%s; Section~\ref{spectroscopy}.)' % oxford_join(names+amalienames)  
caption = caption+r'\label{stellar_props}'
newtab.write('../paper/stellar_props.tex',format='latex',latexdict=latexdict,caption=caption+'\label{stellar_props}')


### Now let's look at the proper spectroscopy with Dnu measurements

In [ ]:
fname_seismic = "../data/smear_Dnu.csv"
seismic = Table.read(fname_seismic)
seismic.remove_row(np.where(seismic['Star_ID']=='HD_189636')[0][0])
seismic.sort('Star_ID')

In [ ]:
seismic

In [ ]:
newtab = Table([Column([str(obj).replace('_',' ') for obj in seismic['Star_ID']],name='Object')])

newkeys = ['Dnu', 'Numax', 'eps']
names = [r'\Dnu',r'\numax', r'$\epsilon$']
units = [r'(\muHz)', r'(\muHz)','']

for k, key in enumerate(newkeys):
    col = seismic[key]
    if key+'_err' in seismic.keys():
        ecol = seismic['%s_err' % key]
        if 'Teff' in key:
            test = Column(['--' if 'nan' in str(col[j]) else '$%.0f \pm %.0f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=names[k])
        else:
            test = Column(['--' if 'nan' in str(col[j]) else '$%.2f \pm %.2f$' % (col[j],ecol[j]) for j, x in enumerate(col)],name=names[k])
    else:
        test = Column(['%.1f' % (col[j]) for j, x in enumerate(col)],name=names[k])
    
    newtab.add_column(test)

latexdict = ascii.latex.latexdicts['AA'].copy()
latexdict['tabletype']= 'table'
latexdict['units'] = dict(zip(names,units))

caption = r'Global asteroseismic parameters %s for the red giant sample as discussed in Section~\ref{asteroseismology}.' % oxford_join(names)  
caption = caption+r'\label{astero_table}'
newtab.write('../paper/astero_table.tex',format='latex',latexdict=latexdict,caption=caption+'\label{stellar_props}')


### Notes on individual stars from Guy Davies

BD+39_388 - No secure detection.

BD+43_306 - Big peaks but not consistent enough with RG pattern.

HD_179959 - Pattern is too complicated - is this two stars??  Literature says no.

HD_185351 - Odd mode envelope - No explanation.

HD_187217 - No info on this star - maybe a double but very difficult to tell.

HD_188639 - Too difficult to get a robust Dnu.

HD_188875 -  Too difficult to get a robust Dnu.

HD_188629 - Too difficult to get a robust Dnu.



In [ ]:
# the stars follow the expected numax-Dnu relation
plt.errorbar(seismic['Numax'],seismic['Dnu'],xerr=seismic['Numax_err'],yerr=seismic['Dnu_err'],linestyle='none')
plt.xlabel(r'$\nu_{max}$ ($\mu$Hz)')
plt.ylabel(r'$\Delta\nu$ ($\mu$Hz)')
plt.title('Asteroseismic Parameters')
plt.xscale('log')
plt.yscale('log')

### Which ones are missing?

In [ ]:
badstars = ['BD+39_388','BD+43_306','HD_179959','HD_185351','HD_187217','HD_188639','HD_188875','HD_188629']
notes = ['No secure detection','Not consistent with RG pattern','Possible contamination',r'\citep{2017MNRAS.464.3713H}',
         'Possible contamination or binary',r'Too difficult to get a robust \Dnu',r'Too difficult to get a robust \Dnu',
         r'Too difficult to get a robust \Dnu']

notecol = [notes[j] if star in badstars else '' for j, star in enumerate(seismic['Star_ID'])]

In [ ]:
np.sort(badstars)

In [ ]:
for star in merged['Object']:
    if star not in seismic['Star_ID']:
        print star

In [ ]:
for star in seismic['Star_ID']:
    if star not in merged['Object']:
        print star

## Read the main catalogue

In [ ]:
fname_cat = '../data/catalogue.csv'
cat = Table.read(fname_cat,format='ascii')
cat.sort('kepmag')

In [ ]:
# cat

In [ ]:
# print 'Clump Candidates:',oxford_join([name.replace('_',' ') for name in cat[np.array(['clump' in thing for thing in list(cat['Notes'].data.data[:])])]['Name']])

### Now to get SIMBAD queries

In [ ]:
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy import units as u


In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('sptype')
sim = Simbad.query_objects(cat['Name'])
cat['SpType'] = sim['SP_TYPE']
sim.write('smear_intermediate.csv')
sim = Table.read('smear_intermediate.csv')
c = coord.SkyCoord(ra=sim['RA'], dec=sim['DEC'],unit=(u.hourangle, u.deg), frame='icrs')


In [ ]:
new = Table([Column([cc.ra.deg for cc in c],name='RA',unit='deg',description='RA'),Column([cc.dec.deg for cc in c],name='Dec',unit='deg',description='Dec')],masked=False)
# new.write('../data/smear_simbad.csv',overwrite=True)

We then use the gaia tap service at http://gaia.ari.uni-heidelberg.de/tap.html for crossmatch 
using the command

SELECT bj.source_id, sm.ra, sm.dec, sm.phot_g_mean_mag, bj.r_est, bj.r_lo, bj.r_hi
FROM gaiadr2_complements.geometric_distance as bj 
JOIN TAP_UPLOAD.smear AS sm ON sm.source_id = bj.source_id

In [ ]:
gaia = Table.read('../data/gaia_new.csv')
gaia[gaia['source_id']==2128480311802353536]

In [ ]:
def abs_mag(app_mag,dist):
    return app_mag - 5.*(np.log10(dist) - 1.)

In [ ]:


dists = Table.read('../data/dists_new.csv')
dists.insert_row(38,
                 vals={'source_id':2128480311802353536,'r_est':np.nan,'r_lo':np.nan,'r_hi':np.nan})
gaia = join(dists,gaia,keys='source_id')

args = np.argsort(new['RA'])
args2 = np.argsort(args)
gaia.sort('ra')
gaia = gaia[args2]


bp, rp, gg = gaia['phot_bp_mean_mag'], gaia['phot_rp_mean_mag'], gaia['phot_g_mean_mag']
abs_gg = abs_mag(gg,dists['r_est'])
gaia['abs_gg'] = abs_gg
gaia['bp_rp'] = bp-rp
abs_gg_hi = abs_mag(gg,dists['r_hi'])
abs_gg_lo = abs_mag(gg,dists['r_lo'])

In [ ]:
# V2083_Cyg doesn't have a parallax
# dists.insert_row(38,
#                  vals={'source_id':2128480311802353536,'ra':292.818076238,'dec':47.481042633,
#                        'phot_g_mean_mag':6.81,'r_est':np.nan,'r_lo':np.nan,'r_hi':np.nan})


In [ ]:
# # # check the RA/Dec are the same
plt.plot(new['RA'],label='Input')
plt.plot(gaia['ra'],label='Gaia')
plt.legend()

In [ ]:
# plt.plot(cat['kepmag'],gaia['phot_g_mean_mag'])

In [ ]:
# cat[np.abs(gaia['phot_g_mean_mag']-cat['kepmag'])>0.75]#.sort(np.abs(dists['phot_g_mean_mag']-cat['kepmag']))

In [ ]:
# gaia[np.abs(gaia['phot_g_mean_mag']-cat['kepmag'])>0.75]#.sort(np.abs(dists['phot_g_mean_mag']-cat['kepmag']))

In [ ]:
# Table(gaia['source_id']).write('sources.csv')

In [ ]:
# newcat = cat.copy()
brightkep = Table.read('../data/Bright_Kep_new.csv')
var = Table.read('../data/variability.csv')
newcat = join(cat,brightkep,keys='Name')
newcat = join(newcat,var,keys='Name')
fname_spc = '../data/spc_giants.csv'
# fname_spc = '../data/spc_final.txt'
spc_data = Table.read(fname_spc,format='ascii')

newcat['TRES?'] = ['TRES' if obj in spc_data['Object'] else '--' for obj in newcat['Name']]
newcat.add_column(gaia['source_id'])

newcat = join(newcat,gaia['bp_rp','source_id','r_est','r_lo','r_hi','phot_g_mean_mag'],keys='source_id',join_type=u'left')

newcat.sort('Name')

In [ ]:
gaia['Name'] = [newcat['Name'][newcat['source_id']==obj][0] for obj in gaia['source_id']]

gaia['TRES'] = [newcat['TRES?'][newcat['source_id']==obj][0] for obj in gaia['source_id']]
pdg = gaia.to_pandas()

In [ ]:
fname_spc2 = '../data/spc_giants.csv'
spc_data2 = Table.read(fname_spc2,format='ascii')

spc_data2

In [ ]:
for l, ccat in enumerate([newcat[:52],newcat[52:]]):

    newtab = Table([Column([str(obj).replace('_',' ') for obj in ccat['Name']],name='Object')])

    newkeys = ['Kepler_ID','SpType','kepmag', 'phot_g_mean_mag','bp_rp','r_est','TRES?','Class']
    names = [r'KIC',r'Spectral Type',r'$Kp$',r'$G$',r'$Bp-Rp$',r'\gaia Distance','TRES','Variability']
    units = ['',r'(SIMBAD)', r'(mag)','(mag)',r'(mag)','(pc)','','Class']
    
    translator_class = {'LPV':'LPV',
                        'RG':'RG',
                        'GD/DS Hybrid':r'$\gamma\,\text{Dor} /\delta\,\text{Sct}$',
                        'GD':r'$\gamma\,\text{Dor}$',
                        'DS':r'$\delta\,\text{Sct}$',
                        '?':'?',
                        '-':r'--',
                        'Hump & Spike':'H+S',
                        'GD/Hump & Spike':r'$\gamma\,\text{Dor}$,\,H+S',
                        'SPB/Hump & Spike':r'SPB,\,H+S',
                        'EV':'EV',
                        'EB':'EB',
                        'SPB':'SPB',
                        'Irregular':'Irregular',
                        'a2cvn':r'$\alpha^2\,\text{CVn}$',
                        'RM':r'RM',
                        'Eruptive':r'Eruptive'
                       }

    for k, key in enumerate(newkeys):
        col = ccat[key]
        if 'RE' in key:
            test = Column(['\checkmark' if 'TRES' in entry else '--' for entry in col],name=names[k])
        elif 'bs' in key:
            test = Column([entry.replace('under','') if 'under' in entry else 'unobserved' for entry in col],name=names[k])
        elif 'ype' in key:
            test = Column(['--' if np.ma.is_masked(entry) else entry for entry in col],name=names[k])
        elif 'est' in key:
            upcol = ccat['r_hi']
            downcol = ccat['r_lo']
            test = Column(['--' if 'nan' in str(col[j]) else '$%.1f^{+%.1f}_{-%.1f}$' % (col[j],upcol[j]-col[j],col[j]-downcol[j]) for j, x in enumerate(col)],name=names[k])
        elif key =='Class':
            test = Column([translator_class[entry] for entry in col],name=names[k])
        elif 'mag' in key:
            test = Column(['%.3f' % d for d in col.data],name=names[k])
        elif 'bp' in key:
            test = Column(['%.3f' % d for d in col.data],name=names[k])
        else:
            test = Column(col.data,name=names[k])
#         if key == 'TRES?':
#             test = Column(['TRES' if newtab['Object'][j].replace(' ','_') in merged['Object'] else '--' for j,entry in enumerate(col)], name=names[k])
        newtab.add_column(test)
    
    latexdict = ascii.latex.latexdicts['AA'].copy()
    latexdict['tabletype']= 'table*'
    latexdict['units'] = dict(zip(names,units))
    if l == 0:
        caption = r"The full set of underobserved and unobserved stars for which new light curves have been produced in this smear catalogue. \
        Calibrated \gaia distances are from \citet{gaiadists}. \
        The eclipsing binary V2083~Cyg was detected by \gaia, but a parallax could not be obtained in DR2, possibly due to binary motion.\
        Variability classes are determined by inspection, having their usual abbreviations. \
        EV denotes an ellipsoidal variable, and RM rotational modulation, though these two can appear similar. $\alpha^2\,\text{CVn}$ variables are chemically-peculiar stars with rotational spot modulation,\
        and are noted separately from RM without chemical peculiarity.\
        $\gamma\,\text{Dor} /\delta\,\text{Sct}$ denotes a $\gamma\,\text{Dor} /\delta\,\text{Sct}$ hybrid, not uncertainty.\
        H+S denotes a `hump and spike' star.\
        Question marks indicate uncertainty, and dashes -- that no significant variability is observed." 
        caption = caption+r'\label{all_stars}'
        newtab.write('../paper/all_stars.tex',format='latex',latexdict=latexdict,caption=caption+'\label{all_stars}')
        print 'Written %s' % '../paper/all_stars.tex'
    else:
        latexdict['preamble'] = '\contcaption{The full set of underobserved and unobserved stars for which new light curves have been produced in this smear catalogue. Calibrated \gaia distances are from \citet{gaiadists}. \label{all_stars_cont}}'
        newtab.write('../paper/all_stars2.tex',format='latex',latexdict=latexdict)
        print 'Written %s' % '../paper/all_stars2.tex'
        

newtab = Table([Column([str(obj).replace('_',' ') for obj in newcat['Name']],name='Object')])

newkeys = ['Kepler_ID','SpType','kepmag', 'phot_g_mean_mag','r_est','Observed', 'TRES?','Class']
names = [r'KIC',r'Spectral Type',r'$Kp$',r'$G$',r'\gaia Distance',r'Observed','Spectroscopy','Variability']
units = ['',r'(SIMBAD)', r'(mag)','(mag)','(pc)','','','Class']

for k, key in enumerate(newkeys):
    col = newcat[key]
    if 'bs' in key:
        test = Column([entry.replace('under','') if 'under' in entry else 'unobserved' for entry in col],name=names[k])
    elif 'ype' in key:
        test = Column(['--' if np.ma.is_masked(entry) else entry for entry in col],name=names[k])
    elif 'est' in key:
        upcol = newcat['r_hi']
        downcol = newcat['r_lo']
        test = Column(['--' if 'nan' in str(col[j]) else '$%.1f^{+%.1f}_{-%.1f}$' % (col[j],upcol[j]-col[j],col[j]-downcol[j]) for j, x in enumerate(col)],name=names[k])
    elif key =='Class':
        test = Column([translator_class[entry] for entry in col],name=names[k])
    elif 'mag' in key:
        test = Column(['%.3f' % d for d in col.data],name=names[k])
    elif 'bp' in key:
        test = Column(['%.3f' % d for d in col.data],name=names[k])
    else:
        test = Column(col.data,name=names[k])
#     if key == 'TRES?':
#         test = Column(['TRES' if newtab['Object'][j].replace(' ','_') in merged['Object'] else '--' for j,entry in enumerate(col)], name=names[k])
    newtab.add_column(test)

latexdict = ascii.latex.latexdicts['AA'].copy()
latexdict['tabletype']= 'table*'
latexdict['units'] = dict(zip(names,units))


In [ ]:
col = ccat['r_est']
upcol = ccat['r_hi']
downcol = ccat['r_lo']
test = Column(['--' if 'nan' in str(col[j]) else '$%.2f^{+%.2f}_{-%.2f}$' % (col[j],upcol[j]-col[j],col[j]-downcol[j]) for j, x in enumerate(col)])


In [ ]:
# for_keith = newcat['Name','ra','dec']
# for_keith.write('star_coords_keith.csv')

In [ ]:
newcat

## Now make an HR diagram

In [ ]:
snrs = dists['r_est']/((dists['r_hi']-dists['r_lo'])/2.)
np.nanmedian(snrs)

In [ ]:
errorbars = np.zeros((2,np.size(abs_gg)))
errorbars[0,:] = abs_gg_hi-abs_gg
errorbars[1,:] = abs_gg-abs_gg_lo

In [ ]:
rc('text', usetex=False)
m = np.where(gaia['TRES']!='--')[0]
plt.errorbar(bp[m]-rp[m],abs_gg[m],yerr=errorbars[:,m],fmt='.',color=colours[1],label=r'Smear Campaign (with spectra)',markersize=10)
plt.errorbar(bp[~m]-rp[~m],abs_gg[~m],yerr=errorbars[:,~m],fmt='.',color=colours[2],label=r'Smear Campaign (no spectra)',markersize=10)
plt.gca().invert_yaxis()
plt.xlabel(r'$Bp-Rp$')
plt.ylabel(r'Absolute $G$')
plt.legend()

Now we want to use Megan Bedell's Gaia-Kepler crossmatch to situate these in the broader colour-magnitude diagram

In [ ]:
kep_data = Table.read('../data/kepler_dr2_4arcsec.fits', format='fits')


In [ ]:
kep_data['abs_gmag'] = kep_data['phot_g_mean_mag'] - \
                            5.*(np.log10(kep_data['r_est']) - 1.)
kep_data['abs_gmag'].unit = u.mag

In [ ]:
newcat

In [ ]:
filter = (kep_data['phot_g_mean_mag'] <= 18.) & (kep_data['parallax'] > 0.1)
filter = (kep_data['phot_g_mean_mag'] <= 18.) & (kep_data['parallax_over_error'] > 25)

plt.scatter(kep_data['bp_rp'][filter], kep_data['abs_gmag'][filter], alpha=0.15, color='k', s=1,label='All Kepler')
m = np.where(gaia['TRES']!='--')[0]
plt.errorbar(bp[m]-rp[m],abs_gg[m],yerr=errorbars[:,m],fmt='.',color=colours[1],label=r'Smear + spectra',markersize=11)
plt.errorbar(bp[~m]-rp[~m],abs_gg[~m],yerr=errorbars[:,~m],fmt='.',color=colours[2],label=r'Smear (no spectra)',markersize=11)

plt.xlabel('Gaia $Bp-Rp$', fontsize=14)
plt.ylabel(r'Absolute $G$ mag', fontsize=14)
plt.ylim([13,-6])
plt.xlim([-0.5,4.1])
plt.legend()
plt.title(r'Smear Stars in the Gaia Colour-Magnitude Diagram',y=1.01,fontsize=14)
# plt.axvline(0.95) # separate giants and dwarfs 
plt.savefig('../paper/gaia_kepler_hr.png',bbox_inches='tight')

In [ ]:
# filter = (kep_data['phot_g_mean_mag'] <= 18.) & (kep_data['parallax'] > 0.1)
# filter = (kep_data['phot_g_mean_mag'] <= 18.) & (kep_data['parallax_over_error'] > 25)

# plt.scatter(kep_data['bp_rp'][filter], kep_data['abs_gmag'][filter], alpha=0.15, color='k', s=1,label='All Kepler')
# m = np.where(gaia['TRES']!='--')[0]
# plt.errorbar(bp[m]-rp[m],abs_gg[m],yerr=errorbars[:,m],fmt='.',color=colours[1],label=r'Smear + spectra',markersize=10)
# plt.errorbar(bp[~m]-rp[~m],abs_gg[~m],yerr=errorbars[:,~m],fmt='.',color=colours[2],label=r'Smear (no spectra)',markersize=10)

# plt.xlabel('Gaia $Bp-Rp$', fontsize=14)
# plt.ylabel(r'Absolute $G$ mag', fontsize=14)
# plt.ylim([13,-6])
# plt.xlim([-0.5,4.1])
# plt.legend()
# plt.title(r'Smear Stars in the Gaia Colour-Magnitude Diagram',y=1.01,fontsize=14)
# # plt.axvline(0.95) # separate giants and dwarfs 
# plt.savefig('../paper/gaia_kepler_hr.png',bbox_inches='tight')

In [ ]:
giants = (bp-rp>0.95)
dwarfs = (bp-rp<0.95)
print giants.sum(),'giants', dwarfs.sum(),'dwarfs',giants.sum()+dwarfs.sum(),'total'
# np.savetxt('giants.txt',cat['Name'][giants])
# np.savetxt('dwarfs.txt',cat['Name'][dwarfs])

In [ ]:
# # find weird stars in the HR diagram
# rc('text', usetex=False)
# plt.errorbar(bp[dwarfs]-rp[dwarfs],abs_gg[dwarfs],yerr=errorbars[:,dwarfs],fmt='.',color=colours[2],label=r'Dwarfs',markersize=10)
# plt.errorbar(bp[giants]-rp[giants],abs_gg[giants],yerr=errorbars[:,giants],fmt='.',color=colours[1],label=r'Giants',markersize=10)

# weird = ['HD_176582', 'HD_179395']
# for j,w in enumerate(weird):
#     ww = np.where(gaia['Name'] == w)[0]
#     plt.errorbar(bp[ww]-rp[ww],abs_gg[ww],yerr=errorbars[:,ww],fmt='.',color=colours[3+j],label=w,markersize=25)

# plt.gca().invert_yaxis()
# plt.xlabel(r'$Bp-Rp$')
# plt.ylabel(r'Absolute $G$')
# plt.legend()
# plt.savefig('hr_weird.png')

In [ ]:
test = Table({'Giants':cat['Name'][giants].data.data.astype('S10')})
test.write('giants.txt',format='ascii',overwrite=True)

test = Table({'dwarfs':cat['Name'][dwarfs].data.data.astype('S10')})
test.write('dwarfs.txt',format='ascii',overwrite=True)

## Now compare to Hypatia

In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('ids')
sim = Simbad.query_objects(cat['Name'])
ids = sim['IDS']

In [ ]:
import re
hips = []
for eyedee in ids:
    m = re.search('\|HIP  (.+?)\|',str(eyedee))
    if m:
        found = m.group(1)
#     print found
    hips.append(found)

In [ ]:
hinkel = Table.read('../data/hypatia-03082018.csv')

In [ ]:
in_hyp = []
for j, hip in enumerate(hips):
    if int(hip) in hinkel['f_hip']:
        print hip, sim['MAIN_ID'][j]
        in_hyp.append((hip,sim['MAIN_ID'][j]))

In [ ]:
in_hyp

In [ ]:
# import altair as alt
import pandas as pd

In [ ]:
pdg = gaia.to_pandas()
pdg.assign(bprp=pdg.phot_bp_mean_mag - pdg.phot_rp_mean_mag);
pdg.insert(len(pdg.columns), 'bprp', pd.Series(pdg.phot_bp_mean_mag - pdg.phot_rp_mean_mag, index=pdg.index))
pdg.insert(len(pdg.columns), 'name', pd.Series(sim['MAIN_ID'], index=pdg.index))
spec = ['TRES' if sp else 'No  Spectra' for sp in pdg['TRES']=='TRES']
pdg.insert(len(pdg.columns), 'Spectroscopy', pd.Series(spec, index=pdg.index))
pdg.insert(len(pdg.columns), 'SpType', pd.Series(newcat['SpType'], index=pdg.index))

# pdkep = kep_data.to_pandas()

In [ ]:
pdg.Spectroscopy

### The *Kepler* dataset is too big for Altair. But not for Bokeh - let's copy from at demo.ipynb from github.com/megbedell/gaia-kepler.fun

In [ ]:
from bokeh.plotting import *
from bokeh.io import output_notebook
from bokeh.models import OpenURL, Circle, HoverTool, PanTool, BoxZoomTool, ResetTool, SaveTool, TapTool, WheelZoomTool
from bokeh.util.hex import hexbin
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import Spectral6
# output_notebook()
reset_output()
output_file("cmd_smear.html")


In [ ]:
filter1 = (kep_data['phot_g_mean_mag'] <= 18.) & (kep_data['parallax_over_error'] > 25)
# filter2 = (kep_data['planet?'] == 'conf') & filter1

filter2 = filter1 

thin = 4

source = ColumnDataSource(
    data=dict(
        kepid=kep_data[filter2]['kepid'].data.data[::thin],
        kepmag=kep_data[filter2]['kepmag'].data.data[::thin],
        bp_rp=kep_data[filter2]['bp_rp'].data.data[::thin],
        abs_gmag=kep_data[filter2]['abs_gmag'].data.data[::thin],
        mass=kep_data[filter2]['mass'].data.data[::thin],
        radius=kep_data[filter2]['radius'].data.data[::thin]
        )
    )

source_smear = ColumnDataSource(
    data=dict(
        name=pdg.name.values,
        abs_gmag=pdg.abs_gg.values,
        bp_rp=pdg.bprp.values,
        spec=pdg.Spectroscopy.values,
        sptype=pdg.SpType.values
        )
    )


In [ ]:
# dict(
#         name=pdg.name.values,
#         abs_gmag=pdg.abs_gg.values,
#         bp_rp=pdg.bprp.values,
#         spec=pdg.Spectroscopy.values,
#         sptype=pdg.SpType.values
#         )

In [ ]:
%%time

xrange = (-0.5,4.1)
yrange = (13,-6)

fig = figure(tools="wheel_zoom,pan,reset",x_range=xrange,
        y_range=yrange,title="Smear Stars in the Gaia Colour-Magnitude Diagram")   
kep_render = fig.circle('bp_rp','abs_gmag', source=source, size=4, name='KICs',color='black',alpha=0.2)

smear_render = fig.circle('bp_rp','abs_gmag', source=source_smear, size=10, name='Smear',
                          color=factor_cmap('spec',palette=Spectral6,factors=pdg.Spectroscopy.values),legend='spec')
hover = HoverTool(renderers=[smear_render],
                    tooltips=[
        ("Name", "@name"),
        ("Sp. Type","@sptype"),
        ("Abs. G mag","@abs_gmag"),
        ("Bp-Rp","@bp_rp")
                            ]
    )
fig.add_tools(hover)
fig.legend.orientation = "vertical"
fig.legend.location = "bottom_left"

fig.xaxis.axis_label = 'Gaia Bp-Rp (mag)'
fig.yaxis.axis_label = 'Gaia Abs. G Mag'
fig.xaxis.axis_label_text_font_size = '14pt'
fig.xaxis.major_label_text_font_size = '12pt'
fig.yaxis.axis_label_text_font_size = '14pt'   
fig.yaxis.major_label_text_font_size = '12pt' 
save(fig)
# show(fig)


In [ ]:
# for j, star in enumerate(pdg.name.values):
#     print j, star, pdg.abs_gg.values[j]

### Let's see if Hipparcos parallaxes differ from Gaia

In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('parallax')
sim = Simbad.query_objects(cat['Name'])
hip_par = sim['PLX_VALUE']


In [ ]:
from astroquery.esasky import ESASky
esasky = ESASky()


In [ ]:
hippar = []
for j, hip in enumerate(hips):
    print j, hip
    dummy = esasky.query_object_catalogs('HIP'+str(hip),catalogs='HIPPARCOS-2')[0]
    hippar.append(dummy['plx'].data.data[0])

In [ ]:
parallaxes_hip = np.array(hippar)

In [ ]:
plt.plot(gaia['phot_g_mean_mag'],gaia['parallax']/parallaxes_hip,'.',markersize=12,label='Gaia/Hipparcos')
plt.axhline(1.,color=colours[1],label='1')
plt.legend()
plt.xlabel(r'Gaia $G$ mag')
plt.ylabel('Parallax Ratio')
plt.title('Comparison of Gaia and Hipparcos Parallaxes',y=1.01)
plt.savefig('gaiavship.png')

In [ ]:
plt.plot(gaia['parallax'],parallaxes_hip,'.')
plt.plot(gaia['parallax'],gaia['parallax'],'-')

In [ ]:
plt.hist(gaia['parallax']/parallaxes_hip,bins=20);
plt.axvline(1,color=colours[2])

### Now let's check everything against the Bright_Kep overall bright kepler catalogue

In [ ]:
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

brightkep = Table.read('../data/Bright_Kep_new.csv')
# reload(sys)


In [ ]:
brightkep.sort('Name')

In [ ]:
test = join(cat,brightkep,keys='Name')
test.sort('Name')
test

In [ ]:
cat

In [ ]:
pdcat, pdbr = cat.to_pandas(), brightkep.to_pandas()

In [ ]:
# pdcat.join(pdbr,on='Name')

### find missing files

In [ ]:
import glob
filenames = (glob.glob("../data/lcs/*.csv"))


In [ ]:
test = glob.glob("../data/lcs/V543_Lyr_smear_combined.csv")
print test

In [ ]:
for name in cat['Name']:
    test = glob.glob("../data/lcs/%s_smear_combined.csv" % name)
#     print name, test

In [ ]:
# brightkep['Kepler_ID'].pprint(max_lines=230)

In [ ]:
names_in = []
for name in cat['Name']:
    if name not in brightkep['Name']:
        print name
        names_in.append(name)
        
print len(names_in)

In [ ]:
# brightkep['Name'].pprint(max_lines=230)

###  Load stellar variability classes

In [ ]:
var = Table.read('../data/variability.csv')

In [ ]:
for name in cat['Name']:
    if name not in var['Name']:
        print name

In [ ]:
newcat[newcat['Name']=='BD+42_3150']

## Apogee

In [ ]:
fname_apogee = '../data/Pope_APOGEEDR14.fits'
apogee = Table.read(fname_apogee)

In [ ]:
for j, name in enumerate(apogee['Name']):
    apogee['Name'][j] = name.replace(' ','')
apogee['Object']=apogee['Name']


In [ ]:
apogee_seismic = join(apogee,astero_logg,keys='Object')
apogee_seismic['Object']

In [ ]:
apogee_var = join(apogee,var,keys='Name')

In [ ]:
apogee_var['Name','Class']

### Make a Venn diagram


In [ ]:
from matplotlib_venn import venn2, venn3, venn3_circles


In [ ]:
seismic_names = set(seismic['Star_ID'])
# seismic_names.remove('HD_189636')
# seismic_names.add('HD_189636A') # typo star make A
spec_names = set(spc_data['Object'][spc_data['Object']!='HD_176466']) # typo star remove
all_names = set(newcat['Name'])
# giants_names = set(cat['Name'][giants].data.data.astype('S10'))
# dwarf_names = set(cat['Name'][dwarfs].data.data.astype('S10'))

In [ ]:
def label_by_id(label, ID):
    num = v.get_label_by_id(ID).get_text() 
    v.get_label_by_id(ID).set_text(label+"\n"+num)


labels = ('Asteroseismology','Spectroscopy','All Stars')


In [ ]:
v = venn3([seismic_names,spec_names,all_names],set_labels=labels,alpha=0.2)
lbl = v.get_label_by_id('001')
x, y = lbl.get_position()
lbl.set_position((x, y-0.1))  # Or whatever
lbl = v.get_label_by_id('111')
x, y = lbl.get_position()
lbl.set_position((x-0.3, y))  # Or whatever


venn3_circles([seismic_names,spec_names,all_names],linewidth=0.5);
plt.savefig('../paper/venn.png',bbox_inches='tight')

In [ ]:
for name in spec_names:
    if name not in all_names:
        print 'Spectroscopy:',name,'not in main catalogue'

for name in seismic_names:
    if name not in all_names:
        print 'Asteroseismology:',name,'not in main catalogue'
        
for name in seismic_names:
    if name not in spec_names:
        print 'Asteroseismic data available for',name,'but not spectroscopy'
        
for name in diff['Object']:
    if name not in seismic_names:
#         print 'Abundances data available for',name,'but not asteroseismology'
        print name


In [ ]:
seismic[seismic['Star_ID']=='HD_189636']

In [ ]:
spc_data['SNRe'].mean()